# Introduction to MNE-NIRS and MNE-Python 

Useful links:

Documentation:
* MNE-Python: https://mne.tools
* MNE-NIRS: https://mne.tools/mne-nirs

Code:
* MNE-Python: hhttps://github.com/mne-tools/mne-python
* MNE-NIRS: https://github.com/mne-tools/mne-nirs

Community:
* User forum: https://mne.discourse.group/
* Chat room: https://discord.com/invite/rKfvxTuATa

Tutorials:
* All fNIRS tutorials: https://mne.tools/mne-nirs/master/auto_examples/index.html

## Outline

This tutorial will cover the following:

* What is MNE and MNE-NIRS?
* Where can I get help using MNE?
* The basics
  * Python basics
  * How to load NIRx data
  * How to view raw data
  * How to manipulate data
    * How to pick channels
    * How to crop data
* Basic signal processing
  * Conversion to haemoglobin
  * Data quality metrics
* Waveform analysis

### Webinar 2: Advanced topics
What should we discuss?
* GLM analysis
* Group level waveform analysis
* Group level GLM analysis
* Cortical projection
* ...
* What do you want to see?

## Python basics

### Packages and functions

In [ ]:
# Importing a package
import mne
import mne_nirs

In [ ]:
# Other ways to import a package
import numpy as np

In [ ]:
# Import just a single function from a package
from itertools import compress

In [ ]:
# Running a function
np.zeros(4)

In [ ]:
# Set defaults using a function
mne.viz.set_3d_backend("notebook")

## How to load NIRx data

In [ ]:
# Import required libraries
import os
import matplotlib.pyplot as plt

In [ ]:
# Download example data and report the path to data
fnirs_data_folder = mne.datasets.fnirs_motor.data_path()
fnirs_data_folder

In [ ]:
# Get path for just the first participant
fname = '/home/mne_user/mne_data/MNE-fNIRS-motor-data/Participant-1'

In [ ]:
# Load the NIRx data
raw_intensity = mne.io.read_raw_nirx(fname, verbose=True, preload=True)

In [ ]:
raw_intensity

## How to view your data

In [ ]:
raw_intensity.plot_sensors();

In [ ]:
events, event_dict = mne.events_from_annotations(raw_intensity, verbose=False)

In [ ]:
event_dict = {'Control': 1, 'Tapping/Left': 4, 'Tapping/Right': 3, 'ExperimentEnds': 2}

In [ ]:
plt.rcParams["figure.figsize"] = (16, 6) # (w, h)
mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_intensity.info['sfreq']);

## Alternative loading of data

In [ ]:
# NIRSport2 can export as SNIRF which can be read using
mne.io.read_raw_snirf("fake+path")

In [ ]:
# Or the best option is to use MNE-BIDS if your data is formatted correctly
from mne_bids import BIDSPath, read_raw_bids
from mne_nirs.datasets import fnirs_motor_group

bids_path = BIDSPath(subject="01", task="tapping",
                     root=fnirs_motor_group.data_path(),
                     datatype="nirs", suffix="nirs", extension=".snirf")

raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
raw_intensity.plot(duration=300, n_channels=len(raw_intensity.ch_names));

## How to manipulate data

In [ ]:
# Working on a copy
raw = raw_intensity.copy()

In [ ]:
raw

In [ ]:
raw.resample(2)

In [ ]:
raw

In [ ]:
raw_intensity

In [ ]:
raw_intensity.annotations

In [ ]:
raw_intensity.annotations.to_data_frame()

In [ ]:
raw_intensity.annotations.to_data_frame().plot.scatter(x='onset',  y='description')

### Picking channels

In [ ]:
raw.copy().pick(picks=[0, 1, 2, 3])

In [ ]:
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)
raw.copy().pick(picks=[0, 1, 2, 3]).plot_sensors();

In [ ]:
plt.rcParams["figure.figsize"] = (16, 6) # (w, h)
raw.copy().pick(picks=[0, 1, 2, 3]).plot(duration=300);

In [ ]:
raw.copy().pick(picks=[10]).plot(duration=300);

In [ ]:
raw.copy().pick(picks=range(13, 19)).plot(duration=300);

### Cropping data

In [ ]:
raw.copy().crop(tmin=50, tmax=100).plot(duration=300);

In [ ]:
raw.copy().pick(range(4)).crop(tmin=50, tmax=100).plot(duration=300);

## Signal Processing

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

In [ ]:
raw_od

In [ ]:
raw_od.copy().pick(range(6)).ch_names

In [ ]:
# Import required functions
from mne.preprocessing.nirs import beer_lambert_law, optical_density

In [ ]:
raw_haemo = beer_lambert_law(raw_od)

In [ ]:
raw_haemo

In [ ]:
raw_haemo.copy().pick(range(6)).ch_names

In [ ]:
# We can now pick based on type
raw_haemo.copy().pick("hbo").ch_names

In [ ]:
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
raw_haemo.plot(duration=300, n_channels=len(raw_haemo.ch_names), clipping=None);

In [ ]:
plt.rcParams["figure.figsize"] = (16, 8) # (w, h)
raw_haemo.copy().pick("hbo").plot(duration=300, n_channels=len(raw_haemo.ch_names), clipping=None);

## Data Quality